In [1]:
import pandas as pd

Open swingers files

In [2]:
swingers_bd = pd.read_csv('../../../data/Swingers BD.csv')
swingers_da = pd.read_csv('../../../data/Swingers DA.csv')
swingers_ba = pd.read_csv('../../../data/Swingers BA.csv')

# print total
print(f"Number of BD swingers: {len(swingers_bd)}")
print(f"Number of DA swingers: {len(swingers_da)}")
print(f"Number of BA swingers: {len(swingers_ba)}")

Number of BD swingers: 5324
Number of DA swingers: 8303
Number of BA swingers: 3699


In [3]:
swingers_bd['swing_time'] = 'BD'
swingers_da['swing_time'] = 'DA'
swingers_ba['swing_time'] = 'BA'

In [4]:
swingers = pd.concat([swingers_bd, swingers_da, swingers_ba])

# Study swing to types

Adjust non-political communities to "noparty"

In [5]:
# find where party1 is nan
swingers["party1"] = swingers["party1"].fillna("noparty").astype(str)
swingers["party2"] = swingers["party2"].fillna("noparty").astype(str)

Open political representative files

In [6]:
representatives = pd.read_csv('../../../data/twitter_representatives_handles_final.csv')

Get unique parties

In [7]:
all_parties = representatives["Party"].unique()
all_parties

array(['Alleanza Verdi Sinistra', 'Azione - Italia Viva', 'Forza Italia',
       "Fratelli d'Italia", 'Lega', 'Movimento 5s', 'Noi moderati',
       'Partito Democratico'], dtype=object)

Specificy party abbrevations and coalition groups

In [8]:
all_parties_abbr = ["AVS", "Az-Iv", "FI", 
                    "FdI", "L", "M5s", "NM",
                    "PD"]

In [9]:
coalition_right = ["FI", "FdI", "L", "NM"]
coalition_left = ["AVS", "PD"]

Populate swing type lists

In [10]:
norealswing = []
nantoparty = []
partytonan = []
soft_swing = []
hard_swing = []

swing_type = []
for row, data in swingers.iterrows():

    if data["party1"]=="noparty":
        nantoparty.append(["noparty", data["party2"]])
        swing_type.append("nantoparty")
    elif data["party2"]=="noparty":
        partytonan.append([data["party1"], "noparty"])
        swing_type.append("partytonan")
    else:
        list_coalition_1 = data["party1"].split(";")
        list_coalition_2 = data["party2"].split(";")
        if data["party1"] in list_coalition_2 or data["party2"] in list_coalition_1 or set(list_coalition_1).intersection(list_coalition_2):
            norealswing.append([data["party1"], data["party2"]])
            swing_type.append("norealswing")
        else:
            # soft swing considering list_coalition_1, list_coalition_2
            if set(list_coalition_1).intersection(coalition_right) and set(list_coalition_2).intersection(coalition_right):
                soft_swing.append([data["party1"], data["party2"]])
                swing_type.append("softswing")
            elif set(list_coalition_1).intersection(coalition_left) and set(list_coalition_2).intersection(coalition_left):
                soft_swing.append([data["party1"], data["party2"]])
                swing_type.append("softswing")
            # hard swing
            else:
                hard_swing.append([data["party1"], data["party2"]])
                swing_type.append("hardswing")

In [11]:
swingers["swing_type"] = swing_type

# save to csv
swingers.to_csv('../../../data/swingers_full.csv', index=False)

In [12]:
swingers["swing_type"].value_counts()

norealswing    11632
nantoparty      3490
hardswing       1708
partytonan       479
softswing         17
Name: swing_type, dtype: int64

## Analyze results and save different types to csv

In [13]:
norealswing = pd.DataFrame(norealswing, columns=["party1", "party2"])
# remove duplicates
norealswing = norealswing.drop_duplicates()
norealswing.to_csv('../../../data/norealswing.csv', index=False)

In [14]:
nantoparty = pd.DataFrame(nantoparty, columns=["party1", "party2"])
# remove duplicates
nantoparty = nantoparty.drop_duplicates()
nantoparty.to_csv('../../../data/nantoparty.csv', index=False)

In [15]:
partytonan = pd.DataFrame(partytonan, columns=["party1", "party2"])
# remove duplicates
partytonan = partytonan.drop_duplicates()
partytonan.to_csv('../../../data/partytonan.csv', index=False)

In [16]:
soft_swing = pd.DataFrame(soft_swing, columns=["party1", "party2"])
# remove duplicates
soft_swing = soft_swing.drop_duplicates()
soft_swing.to_csv('../../../data/soft_swing.csv', index=False)

In [17]:
hard_swing = pd.DataFrame(hard_swing, columns=["party1", "party2"])
# remove duplicates
hard_swing = hard_swing.drop_duplicates()
hard_swing.to_csv('../../../data/hard_swing.csv', index=False)